# Setup Walkthrough: Getting Your Environment Ready

Welcome! This notebook will guide you through setting up your development environment for building transformers.

## What We'll Do

1. ✓ Verify Python version (3.11+)
2. ✓ Install PyTorch with appropriate backend (CUDA/MPS/CPU)
3. ✓ Install course dependencies
4. ✓ Test your setup with example code
5. ✓ Run environment validation

**Estimated time:** 15-20 minutes

## Step 1: Verify Python Version

We require Python 3.11 or higher for this course.

In [ ]:
import sys

print(f"Python version: {sys.version}")
print(f"Version info: {sys.version_info}")

# Check minimum version
if sys.version_info >= (3, 11):
    print("\n✓ Python version is compatible!")
else:
    print("\n❌ Please upgrade to Python 3.11 or higher")
    print("   Visit: https://www.python.org/downloads/")

## Step 2: Check Platform and Available Hardware

Let's see what hardware you have available for training.

In [ ]:
import platform

print(f"Operating System: {platform.system()}")
print(f"Platform: {platform.platform()}")
print(f"Processor: {platform.processor()}")
print(f"Architecture: {platform.machine()}")

# Platform-specific info
if platform.system() == "Darwin":  # macOS
    print("\nRunning on macOS - we'll use Metal Performance Shaders (MPS) if available")
elif platform.system() == "Linux":
    print("\nRunning on Linux - we'll check for CUDA GPUs")
elif platform.system() == "Windows":
    print("\nRunning on Windows - we'll check for CUDA GPUs")

## Step 3: Install PyTorch

**Before running this cell**, you should install PyTorch from the command line:

### macOS (Apple Silicon - M1/M2/M3/M4)
```bash
pip install torch torchvision torchaudio
```

### Linux/Windows with NVIDIA GPU
Visit [pytorch.org](https://pytorch.org/get-started/locally/) and use their selector.

### CPU-only (any platform)
```bash
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
```

After installing, run this cell to verify:

In [ ]:
try:
    import torch
    print(f"✓ PyTorch version: {torch.__version__}")
    print(f"  Built with CUDA: {torch.cuda.is_available()}")
    
    # Check for CUDA
    if torch.cuda.is_available():
        print(f"  CUDA version: {torch.version.cuda}")
        print(f"  GPU devices: {torch.cuda.device_count()}")
        print(f"  GPU 0: {torch.cuda.get_device_name(0)}")
    
    # Check for MPS (Apple Silicon)
    if hasattr(torch.backends, 'mps'):
        print(f"  MPS available: {torch.backends.mps.is_available()}")
        if torch.backends.mps.is_available():
            print("  ✓ Metal Performance Shaders (MPS) enabled!")
    
    # Determine device
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"\n✓ Will use device: CUDA (GPU)")
    elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
        device = torch.device("mps")
        print(f"\n✓ Will use device: MPS (Apple Silicon GPU)")
    else:
        device = torch.device("cpu")
        print(f"\n✓ Will use device: CPU")
    
    print(f"\nDevice set to: {device}")
    
except ImportError:
    print("❌ PyTorch not installed!")
    print("   Please install PyTorch following the instructions above.")

## Step 4: Install Course Dependencies

Install all required packages from the project root:

```bash
# From the project root directory
pip install -e .
```

Or using make:
```bash
make install
```

After installing, verify the installation:

In [ ]:
# Test imports
print("Testing imports...\n")

try:
    import tiny_transformer
    print(f"✓ tiny_transformer version: {tiny_transformer.__version__}")
except ImportError as e:
    print(f"❌ Failed to import tiny_transformer: {e}")

try:
    from tiny_transformer import attention
    print("✓ attention module imported")
except ImportError as e:
    print(f"❌ Failed to import attention: {e}")

try:
    from tiny_transformer.utils import check_shape, plot_attention_pattern
    print("✓ utils module imported")
except ImportError as e:
    print(f"❌ Failed to import utils: {e}")

try:
    import numpy as np
    print(f"✓ numpy version: {np.__version__}")
except ImportError:
    print("❌ numpy not installed")

try:
    import matplotlib
    print(f"✓ matplotlib version: {matplotlib.__version__}")
except ImportError:
    print("❌ matplotlib not installed")

print("\nAll core dependencies are installed!")

## Step 5: Test Your Setup with Example Code

Let's run a simple attention mechanism to verify everything works!

In [ ]:
import torch
from tiny_transformer.attention import scaled_dot_product_attention

# Determine device
if torch.cuda.is_available():
    device = torch.device("cuda")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f"Running on device: {device}\n")

# Create sample inputs
batch_size = 2
seq_len = 4
d_k = 8

q = torch.randn(batch_size, seq_len, d_k, device=device)
k = torch.randn(batch_size, seq_len, d_k, device=device)
v = torch.randn(batch_size, seq_len, d_k, device=device)

print(f"Query shape: {q.shape}")
print(f"Key shape: {k.shape}")
print(f"Value shape: {v.shape}")

# Run attention
output, attention_weights = scaled_dot_product_attention(q, k, v)

print(f"\nOutput shape: {output.shape}")
print(f"Attention weights shape: {attention_weights.shape}")

# Verify attention weights sum to 1
attn_sum = attention_weights.sum(dim=-1)
print(f"\nAttention weights sum (should be ~1.0): {attn_sum[0, 0].item():.6f}")

print("\n✓ Attention mechanism working correctly!")

## Step 6: Run Environment Validation Tool

Our project includes a comprehensive validation tool. Run it from the command line:

```bash
make verify
```

Or directly:
```bash
python tools/verify_environment.py
```

You can also run it from this notebook:

In [ ]:
# Run verification tool
import subprocess
import sys

result = subprocess.run(
    [sys.executable, "../../../tools/verify_environment.py"],
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

## Step 7: Run Test Suite

Let's verify the existing test suite passes:

In [ ]:
# Run pytest on the attention module
result = subprocess.run(
    [sys.executable, "-m", "pytest", "../../../tests/test_attention.py", "-v"],
    capture_output=True,
    text=True
)

print(result.stdout)
if result.stderr:
    print("Errors:", result.stderr)

## Step 8: Quick Visualization Test

Let's test our visualization utilities:

In [ ]:
import matplotlib.pyplot as plt
from tiny_transformer.utils import plot_attention_pattern

# Create sample attention pattern
batch_size = 1
seq_len = 8
d_k = 16

q = torch.randn(batch_size, seq_len, d_k)
k = torch.randn(batch_size, seq_len, d_k)
v = torch.randn(batch_size, seq_len, d_k)

output, attention_weights = scaled_dot_product_attention(q, k, v)

# Plot
plot_attention_pattern(
    attention_weights[0],  # First batch
    title="Sample Attention Pattern"
)
plt.show()

print("✓ Visualization working!")

## Success Criteria

You're ready to move forward if:

- [ ] Python 3.11+ is installed
- [ ] PyTorch is installed and working on your device (CUDA/MPS/CPU)
- [ ] All course dependencies imported successfully
- [ ] Attention mechanism example runs without errors
- [ ] Verification tool passes all checks
- [ ] Test suite passes
- [ ] Visualization displays correctly

If all checks pass: **🎉 Congratulations! Your environment is ready!**

## Troubleshooting

### PyTorch Installation Issues
- For macOS: Make sure you're using the latest pip version: `pip install --upgrade pip`
- For CUDA: Visit pytorch.org and use their installation selector
- For CPU-only: Use the `--index-url` flag as shown above

### Import Errors
- Make sure you ran `pip install -e .` from the project root
- Check you're in the correct Python environment (virtual environment)
- Try: `pip install -e . --force-reinstall`

### Test Failures
- Some tests may fail if PyTorch isn't installed correctly
- Check the error messages carefully - they usually indicate what's wrong

## Next Steps

1. ✓ Read `shape_debugging_primer.md` to master shape debugging
2. ✓ (Optional) Review `pytorch_refresher.ipynb` if you need PyTorch review
3. ✓ Move on to Module 01: Attention Mechanism

**You're all set! Happy learning!**